In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"

In [ ]:
import torch
from vllm import LLM, SamplingParams
from vllm.attention.backends.abstract import AttentionType, SharedSelfAttentionType

/home/k/miniconda3/envs/llm_quant/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-09 16:43:03,115	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


### Tests

In [ ]:
# enforce xformers attention backend - changes are made here
os.environ["VLLM_ATTENTION_BACKEND"] = "XFORMERS_CLA"

```python
# Inside LlamaAttention.forward
# Change q to fixed input for testing.
if self.cache_config.debug_kv_sharing: q = torch.ones_like(q)
attn_output = self.attn(q, k, v, kv_cache, attn_metadata, compute_new_kv=compute_new_kv_map)
if self.cache_config.debug_kv_sharing: self.attn_outputs.append(attn_output)
```

### TEST 1

In [ ]:
### test 1: No prefix caching, no quantization, use v1 block manager, no cuda graphs ###
ENABLE_PREFIX_CACHING = False
KV_CACHE_DTYPE = "auto"
USE_V2_BLOCK_MANAGER = True
ENFORCE_EAGER = True
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1, 
                         dtype="bfloat16", 
                         max_model_len=1024*1, 
                         kv_cache_map={0:0, 1:0}, 
                         gpu_memory_utilization=0.2,
                         debug_kv_sharing=DEBUG_KV_SHARING)

WARNING 10-09 16:43:17 config.py:365] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-09 16:43:17 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forw

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-09 16:43:18 selector.py:122] Using XFormers CLA backend.


/home/k/miniconda3/envs/llm_quant/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/k/miniconda3/envs/llm_quant/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 10-09 16:43:19 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...
INFO 10-09 16:43:19 selector.py:122] Using XFormers CLA backend.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.32it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.32it/s]



INFO 10-09 16:43:20 model_runner.py:1060] Loading model weights took 2.8008 GB
INFO 10-09 16:43:20 gpu_executor.py:122] # GPU blocks: 5937, # CPU blocks: 65536


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 37.02it/s, est. speed input: 111.35 toks/s, output: 37.11 toks/s]


In [ ]:
# 3 tokens decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
for attn_output_layer0, attn_output_layer1 in zip(model.model.layers[0].self_attn.attn_outputs, 
                                                  model.model.layers[1].self_attn.attn_outputs):
    assert torch.equal(attn_output_layer0, attn_output_layer1)

# 3 attention metadata for a 3 model forward pass (1 prefill + 2 decode)
assert len(model.model.attn_metadatas) == 3 
# Prefill phase.
attn_metadata = model.model.attn_metadatas[0]
assert attn_metadata.decode_metadata is None
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected
# Decode phase 1.
attn_metadata = model.model.attn_metadatas[1]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected
# Decode phase 2.
attn_metadata = model.model.attn_metadatas[2]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected

Processed prompts: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 20.58it/s, est. speed input: 61.84 toks/s, output: 61.83 toks/s]


### TEST 2 (fp8)

In [ ]:
### test 1: No prefix caching, fp8 quantization, use v1 block manager, no cuda graphs ###
ENABLE_PREFIX_CACHING = False
KV_CACHE_DTYPE = "fp8"
USE_V2_BLOCK_MANAGER = True
ENFORCE_EAGER = True
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1, 
                         dtype="bfloat16",
                         max_model_len=1024*1, 
                         kv_cache_map={0:0, 1:0}, 
                         gpu_memory_utilization=0.2, 
                         debug_kv_sharing=DEBUG_KV_SHARING)

INFO 10-09 16:43:32 config.py:629] Using fp8 data type to store kv cache. It reduces the GPU memory footprint and boosts the performance. Meanwhile, it may cause accuracy drop without a proper scaling factor
WARNING 10-09 16:43:32 config.py:365] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-09 16:43:32 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=fp

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-09 16:43:33 selector.py:122] Using XFormers CLA backend.
INFO 10-09 16:43:34 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...
INFO 10-09 16:43:34 selector.py:122] Using XFormers CLA backend.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.36it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.35it/s]



INFO 10-09 16:43:34 model_runner.py:1060] Loading model weights took 2.7696 GB
INFO 10-09 16:43:35 gpu_executor.py:122] # GPU blocks: 16610, # CPU blocks: 131072


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 159.14it/s, est. speed input: 482.14 toks/s, output: 160.61 toks/s]


AssertionError: 

In [ ]:
# Need to fix this mismatch
attn_output_layer0, attn_output_layer1

(tensor([[ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         [ 0.0013,  0.0067,  0.0334,  ...,  0.0557, -0.0334,  0.0114],
         [ 0.0029,  0.0021,  0.0306,  ...,  0.0140, -0.0356,  0.0200],
         [ 0.0019,  0.0014,  0.0311,  ...,  0.0135, -0.0352,  0.0198],
         [ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         [ 0.0013,  0.0067,  0.0334,  ...,  0.0557, -0.0334,  0.0114]],
        device='cuda:0', dtype=torch.bfloat16),
 tensor([[ 0.0137, -0.0078,  0.0293,  ...,  0.0078, -0.0088,  0.0078],
         [ 0.0031,  0.0021,  0.0339,  ...,  0.0564, -0.0339,  0.0134],
         [ 0.0047, -0.0020,  0.0312,  ...,  0.0142, -0.0369,  0.0204],
         [ 0.0035, -0.0027,  0.0317,  ...,  0.0135, -0.0366,  0.0203],
         [ 0.0137, -0.0078,  0.0293,  ...,  0.0078, -0.0088,  0.0078],
         [ 0.0031,  0.0021,  0.0339,  ...,  0.0564, -0.0339,  0.0134]],
        device='cuda:0', dtype=torch.bfloat16))

### TEST 3 (cuda graphs)

In [ ]:
### test 1: No prefix caching, bf16 quantization, use v1 block manager, cuda graph mode ###
ENABLE_PREFIX_CACHING = False
KV_CACHE_DTYPE = "auto"
USE_V2_BLOCK_MANAGER = True
ENFORCE_EAGER = False
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1, 
                         dtype="bfloat16", 
                         max_model_len=1024*1, 
                         kv_cache_map={0:0, 1:0}, 
                         gpu_memory_utilization=0.2,
                         debug_kv_sharing=DEBUG_KV_SHARING)

INFO 10-09 16:44:55 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=/home/k/models/dummy-llama, use_v2_block_manager=True, num_scheduler_steps=1, chunked_prefill_

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-09 16:44:57 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.22it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.21it/s]



INFO 10-09 16:44:58 model_runner.py:1060] Loading model weights took 2.7696 GB
INFO 10-09 16:44:58 gpu_executor.py:122] # GPU blocks: 8049, # CPU blocks: 65536
INFO 10-09 16:44:58 model_runner.py:1383] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 10-09 16:44:58 model_runner.py:1387] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 10-09 16:45:31 model_runner.py:1511] Graph capturing finished in 33 secs.


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 163.54it/s, est. speed input: 495.73 toks/s, output: 165.06 toks/s]


In [ ]:
# 3 tokens decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
for attn_output_layer0, attn_output_layer1 in zip(model.model.layers[0].self_attn.attn_outputs, 
                                                  model.model.layers[1].self_attn.attn_outputs):
    assert torch.equal(attn_output_layer0, attn_output_layer1)


    
#     NOTE: Any python object stored here is not updated when it is
#     cuda-graph replayed. If you have values that need to be changed
#     dynamically, it should be stored in tensor. The tensor has to be
#     updated from `CUDAGraphRunner.forward` API.
    
    
# # 3 attention metadata for a 3 model forward pass (1 prefill + 2 decode)
# assert len(model.model.attn_metadatas) == 3 
# # Prefill phase.
# attn_metadata = model.model.attn_metadatas[0]
# assert attn_metadata.decode_metadata is None
# expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
# assert attn_metadata.prefill_metadata.shared_self_attention_types == expected
# # Decode phase 1.
# attn_metadata = model.model.attn_metadatas[1]
# assert attn_metadata.prefill_metadata is None
# expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
# assert attn_metadata.decode_metadata.shared_self_attention_types == expected
# # Decode phase 2.
# attn_metadata = model.model.attn_metadatas[2]
# assert attn_metadata.prefill_metadata is None
# expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
# assert attn_metadata.decode_metadata.shared_self_attention_types == expected

Processed prompts: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 79.58it/s, est. speed input: 239.93 toks/s, output: 239.80 toks/s]


### TEST 4 (block manager v1)

In [ ]:
ENABLE_PREFIX_CACHING = False
KV_CACHE_DTYPE = "auto"
USE_V2_BLOCK_MANAGER = False
ENFORCE_EAGER = True
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1, 
                         dtype="bfloat16", 
                         max_model_len=1024*1,
                         kv_cache_map={0:0, 1:0}, 
                         gpu_memory_utilization=0.2,
                         debug_kv_sharing=DEBUG_KV_SHARING)

WARNING 10-09 16:47:27 config.py:365] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-09 16:47:27 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forw

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-09 16:47:29 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.29it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.28it/s]



INFO 10-09 16:47:29 model_runner.py:1060] Loading model weights took 2.7696 GB
INFO 10-09 16:47:30 gpu_executor.py:122] # GPU blocks: 8049, # CPU blocks: 65536


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 121.16it/s, est. speed input: 366.24 toks/s, output: 121.98 toks/s]


In [ ]:
# 3 tokens decode.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=3, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
for attn_output_layer0, attn_output_layer1 in zip(model.model.layers[0].self_attn.attn_outputs, 
                                                  model.model.layers[1].self_attn.attn_outputs):
    assert torch.equal(attn_output_layer0, attn_output_layer1)

# 3 attention metadata for a 3 model forward pass (1 prefill + 2 decode)
assert len(model.model.attn_metadatas) == 3 
# Prefill phase.
attn_metadata = model.model.attn_metadatas[0]
assert attn_metadata.decode_metadata is None
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected
# Decode phase 1.
attn_metadata = model.model.attn_metadatas[1]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected
# Decode phase 2.
attn_metadata = model.model.attn_metadatas[2]
assert attn_metadata.prefill_metadata is None
expected = [SharedSelfAttentionType.DECODE_KV_NEW.name, SharedSelfAttentionType.DECODE_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.decode_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 113.05it/s, est. speed input: 342.08 toks/s, output: 341.75 toks/s]


### TEST 5 (prefix caching)

In [ ]:
### test 1: No prefix caching, bf16 quantization, use v1 block manager, cuda graph mode ###
ENABLE_PREFIX_CACHING = True
KV_CACHE_DTYPE = "auto"
USE_V2_BLOCK_MANAGER = False
ENFORCE_EAGER = True
DEBUG_KV_SHARING = True


# This the base llm generator with KV cache sharing, where layer 1 gets its KV cache from layer 0.
test_llm_generator = LLM(model="/home/k/models/dummy-llama", 
                         tokenizer="meta-llama/Meta-Llama-3.1-8B-Instruct", 
                         enforce_eager=ENFORCE_EAGER, 
                         kv_cache_dtype=KV_CACHE_DTYPE,
                         enable_prefix_caching=ENABLE_PREFIX_CACHING, 
                         use_v2_block_manager=USE_V2_BLOCK_MANAGER,
                         tensor_parallel_size=1,
                         dtype="bfloat16",
                         max_model_len=1024*1,
                         kv_cache_map={0:0, 1:0},
                         gpu_memory_utilization=0.2,
                         debug_kv_sharing=DEBUG_KV_SHARING)

WARNING 10-09 16:48:41 config.py:365] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 10-09 16:48:41 llm_engine.py:237] Initializing an LLM engine (v0.1.dev2968+g2fa3c83) with config: model='/home/k/models/dummy-llama', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=1024, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forw

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'
ERROR:huggingface_hub.file_download:Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/k/.cache/huggingface/hub/models--meta-llama--Meta-Llama-3.1-8B-Instruct/.no_exist/0e9e39f249a16976918f6564b8830bc894c89659/tokenizer.model'


INFO 10-09 16:48:42 model_runner.py:1049] Starting to load model /home/k/models/dummy-llama...


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.31it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  2.31it/s]



INFO 10-09 16:48:43 model_runner.py:1060] Loading model weights took 2.7696 GB
INFO 10-09 16:48:43 gpu_executor.py:122] # GPU blocks: 8049, # CPU blocks: 65536
INFO 10-09 16:48:43 block_manager_v1.py:263] Automatic prefix caching is enabled.


In [ ]:
model = test_llm_generator.llm_engine.model_executor.driver_worker.model_runner.model
worker = test_llm_generator.llm_engine.model_executor.driver_worker

In [ ]:
# Debugging is on.
assert model.model.cache_config.debug_kv_sharing

In [ ]:
def reset_attn_outputs_and_metadatas(model):
    # Reset attn outputs and attn metadatas from warmup.
    if len(model.model.layers[0].self_attn.attn_outputs) > 0:
        model.model.layers[0].self_attn.attn_outputs = []
    if len(model.model.layers[1].self_attn.attn_outputs) > 0:
        model.model.layers[1].self_attn.attn_outputs = []
    if len(model.model.attn_metadatas) > 0:
        model.model.attn_metadatas = []

In [ ]:
# Just prefill.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

# Attention outputs of layer 0 and layer 1 should be equal, Q is fixed to 1s during debug mode.
attn_output_layer0 = model.model.layers[0].self_attn.attn_outputs[0]
attn_output_layer1 = model.model.layers[1].self_attn.attn_outputs[0]
assert torch.equal(attn_output_layer0, attn_output_layer1)
# 1 attention metadata for a single model forward pass
assert len(model.model.attn_metadatas) == 1 
attn_metadata = model.model.attn_metadatas[0]
# No decode request with max_tokens=1.
assert attn_metadata.decode_metadata is None
# Layer 0 creates new KV and layer 1 reuses KV from cache during prefill.
expected = [SharedSelfAttentionType.PREFILL_KV_NEW.name, SharedSelfAttentionType.PREFILL_KV_SHARED.name] # [layer 0, layer 1]
assert attn_metadata.prefill_metadata.shared_self_attention_types == expected

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 125.56it/s, est. speed input: 380.40 toks/s, output: 126.66 toks/s]


In [ ]:
# Just prefill with prefix cached.
reset_attn_outputs_and_metadatas(model)
output = test_llm_generator.generate(["hello world!", "hello"], SamplingParams(temperature=0.0, max_tokens=1, ignore_eos=True))

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 114.58it/s, est. speed input: 346.28 toks/s, output: 115.34 toks/s]


In [ ]:
model.model.layers[0].self_attn.attn_outputs

[tensor([[ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         [ 0.0013,  0.0067,  0.0334,  ...,  0.0557, -0.0334,  0.0114],
         [ 0.0029,  0.0021,  0.0306,  ...,  0.0140, -0.0356,  0.0200],
         [ 0.0019,  0.0014,  0.0311,  ...,  0.0135, -0.0352,  0.0198],
         [ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         [ 0.0013,  0.0067,  0.0334,  ...,  0.0557, -0.0334,  0.0114]],
        device='cuda:0', dtype=torch.bfloat16)]

In [ ]:
model.model.layers[1].self_attn.attn_outputs

[tensor([[ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         [ 0.0013,  0.0067,  0.0334,  ...,  0.0557, -0.0334,  0.0114],
         [ 0.0029,  0.0021,  0.0306,  ...,  0.0140, -0.0356,  0.0200],
         [ 0.0019,  0.0014,  0.0311,  ...,  0.0135, -0.0352,  0.0198],
         [ 0.0109, -0.0005,  0.0287,  ...,  0.0007, -0.0010,  0.0004],
         [ 0.0013,  0.0067,  0.0334,  ...,  0.0557, -0.0334,  0.0114]],
        device='cuda:0', dtype=torch.bfloat16)]

In [ ]:
attn_metadata.prefill_metadata.shared_self_attention_types

['PREFILL_KV_NEW', 'PREFILL_KV_SHARED']

In [ ]:
attn_metadata.prefill_metadata.block_tables

tensor([], device='cuda:0', size=(2, 0), dtype=torch.int32)